# ProyectoDSParteI_Hamburg

**Abstracto**  
Este trabajo realiza un análisis descriptivo del dataset **XWines** (muestras de vino con variables fisicoquímicas y una evaluación de calidad). El objetivo es construir una primera narrativa exploratoria que permita entender patrones básicos y validar hipótesis simples, utilizando exclusivamente contenidos vistos: Pandas para lectura y manipulación tabular, NumPy para cálculos numéricos básicos, Matplotlib/Seaborn para visualización (histogramas, boxplots, dispersión, heatmap) y estadística descriptiva (medidas de resumen, correlación).

Las variables incluyen medidas como acidez fija y volátil, ácido cítrico, azúcar residual, cloruros, dióxido de azufre libre y total, densidad, pH, sulfatos, alcohol y una calificación global de calidad. Se busca responder: (i) ¿qué atributos se asocian más con la **calidad**?, (ii) ¿existen relaciones fuertes entre variables estructurales como **azúcar residual** y **densidad**?, (iii) ¿hay evidencias de outliers que ameriten tratamiento en etapas posteriores?, y (iv) ¿hay datos ausentes que condicionen el análisis?

Hipótesis de interés: **H1**: *alcohol* se asocia positivamente con *quality*; **H2**: *volatile_acidity* se asocia negativamente con *quality*; **H3**: *sulphates* y *total_sulfur_dioxide* muestran asociación positiva moderada con *quality* por efectos antioxidantes; **H4**: *residual_sugar* y *density* presentan correlación positiva alta. El flujo realiza: carga del CSV desde GitHub, revisión de dtypes y nulos, resúmenes con `describe`, visualizaciones univariadas y bivariadas, un gráfico multivariado (color por calidad y tamaño por sulfatos), un mapa de correlaciones y detección de outliers con IQR.

El resultado esperado es identificar señales consistentes: mayor **alcohol** y menor **acidez volátil** en vinos de mayor **calidad**, y una relación estructural entre **azúcar residual** y **densidad**. Estas conclusiones orientan pasos siguientes (modelado supervisado básico) manteniendo interpretabilidad y buenas prácticas visuales.


## Preguntas e hipótesis

- **P1:** ¿Mayor alcohol implica mayor calidad? → **H1:** relación positiva.  
- **P2:** ¿La acidez volátil degrada la calidad? → **H2:** relación negativa.  
- **P3:** ¿El total de SO₂ y los sulfatos se asocian a mejor calidad? → **H3:** efecto positivo moderado.  
- **P4:** ¿La densidad se explica por el azúcar residual? → **H4:** relación positiva alta.


In [ ]:
!wget -q -O XWines.csv https://raw.githubusercontent.com/gerardohamburg/CoderHouse/main/XWines.csv

import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
df = pd.read_csv("XWines.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("/", "_", regex=False)


In [ ]:
df.shape, df.dtypes

In [ ]:
na = df.isna().sum().to_frame("faltantes")
na.T

In [ ]:
df.describe().T

In [ ]:
df["quality"].value_counts().sort_index()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 7))
sns.histplot(df["alcohol"], kde=True, ax=axs[0,0])
axs[0,0].set_title("Alcohol")
sns.histplot(df["volatile_acidity"], kde=True, ax=axs[0,1])
axs[0,1].set_title("Acidez volátil")
sns.histplot(df["residual_sugar"], kde=True, ax=axs[1,0])
axs[1,0].set_title("Azúcar residual")
sns.histplot(df["sulphates"], kde=True, ax=axs[1,1])
axs[1,1].set_title("Sulfatos")
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
sns.boxplot(data=df, x="quality", y="alcohol", ax=axs[0])
axs[0].set_title("Alcohol vs Calidad")
sns.boxplot(data=df, x="quality", y="volatile_acidity", ax=axs[1])
axs[1].set_title("Acidez volátil vs Calidad")
sns.scatterplot(data=df, x="residual_sugar", y="density", ax=axs[2])
axs[2].set_title("Azúcar residual vs Densidad")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(7,5))
sns.scatterplot(data=df, x="alcohol", y="volatile_acidity", hue="quality", size="sulphates", sizes=(20,120), alpha=0.8)
plt.title("Alcohol vs Acidez volátil | color=Calidad, tamaño=Sulfatos")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.0)
plt.tight_layout()
plt.show()

In [ ]:
corr = df.corr(numeric_only=True)
plt.figure(figsize=(9,7))
sns.heatmap(corr, cmap="RdBu_r", center=0, annot=False)
plt.title("Mapa de correlaciones")
plt.tight_layout()
plt.show()
corr["quality"].sort_values(ascending=False)

In [ ]:
def iqr_bounds(s):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    return q1 - 1.5*iqr, q3 + 1.5*iqr

outs = {}
for col in ["volatile_acidity", "alcohol"]:
    low, high = iqr_bounds(df[col])
    outs[col] = df[(df[col] < low) | (df[col] > high)][col].shape[0]
outs

## Conclusiones

- No se detectan valores ausentes relevantes si la tabla `na` arroja ceros.  
- La distribución de **quality** suele concentrarse en valores medios.  
- Los histogramas y boxplots sugieren: **alcohol** aumenta con **quality** y **volatile_acidity** disminuye.  
- **residual_sugar** y **density** muestran una relación positiva fuerte.  
- El gráfico multivariado apoya las hipótesis H1–H3: mayor **alcohol**, menor **acidez volátil** y niveles moderados de **sulphates** aparecen en vinos de mayor **calidad**.  
- El heatmap identifica a **alcohol** como correlación positiva más alta con **quality**, y a **volatile_acidity** como la más negativa.  
- Con IQR se identifican outliers en acidez volátil; conviene revisarlos antes de cualquier modelado supervisado.

Este notebook cumple: abstract inicial, preguntas/hipótesis, visualizaciones univariadas/bivariadas/multivariadas, resúmenes numéricos, diagnóstico de nulos y detección básica de outliers, respetando únicamente contenidos del curso.
